# Neighborly Functions App - Cloud Developer With MS Azure - Udacity Project II

## 1. Add configuration

In [ ]:
%run variables.sh

## 2. Create Resources

### 2.1. Login Azure CLI

In [ ]:
! az login

### 2.2. Create Resource Group

In [ ]:
! az group create \
    --name $resourceGroup \
    --location $location

### 2.3. Create Storage Account

In [ ]:
! az storage account create \
    --name $storageAccountName \
    --resource-group $resourceGroup \
    --location $location \
    --sku $storageSku

### 2.4. Create Function App

In [ ]:
! az functionapp create \
    --name $functionName  \
    --storage-account $storageAccountName \
    --consumption-plan-location $location \
    --resource-group $resourceGroup \
    --functions-version $functionVersion \
    --os-type $functionOsType \
    --runtime $functionRuntime

### 2.5. Create a CosmosDB manage MongoDB instance

#### 2.5.1. Create CosmosDB

In [ ]:
! az cosmosdb create \
    -n $cosmosdbAccountName \
    -g $resourceGroup \
    --kind $cosmosdbKind \
    --server-version $cosmosdbServerSersion \
    --default-consistency-level $cosmosdbDefaultConsistencyLevel \
    --enable-automatic-failover $cosmosdbEnableAutomaticFailover

#### 2.5.2. Get CosmosDB Connection String

In [ ]:
! az cosmosdb list-connection-strings --name $cosmosdbAccountName --resource-group $resourceGroup

In [ ]:
! az cosmosdb keys list \
    --type connection-strings \
    --name $cosmosdbAccountName \
    --resource-group $resourceGroup \
    --query "connectionStrings[0].connectionString" \
    --output tsv

1. Get the above connection string.
2. Append it to the variables.sh file.
3. Re-run the following script to update variables

In [ ]:
%run variables.sh

#### 2.5.3. Create MongoDB Collections

In [ ]:
! printf '[{"key": {"keys": ["_id"]}}]' > idxpolicy-congdinh2023.json

1. Create advertisements collection

In [ ]:
! az cosmosdb mongodb collection create \
    -a $cosmosdbAccountName \
    -g $resourceGroup \
    -d $cosmosdbDatabaseName \
    -n advertisements \
    --shard _id \
    --throughput 400 \
    --idx @idxpolicy-congdinh2023.json

2. Create posts collection

In [ ]:
! az cosmosdb mongodb collection create \
    -a $cosmosdbAccountName \
    -g $resourceGroup \
    -d $cosmosdbDatabaseName \
    -n posts \
    --shard _id \
    --throughput 400 \
    --idx @idxpolicy-congdinh2023.json

3. If you want to delete a collection, use:

In [ ]:
! az cosmosdb mongodb collection delete \
    -a $cosmosdbAccountName \
    -d $cosmosdbDatabaseName \
    -n posts \
    -g azure-dev-udacity-prj2 \
    --yes

In [ ]:
! az cosmosdb mongodb collection delete \
    -a $cosmosdbAccountName \
    -d $cosmosdbDatabaseName \
    -n advertisements \
    -g azure-dev-udacity-prj2 \
    --yes

4. View the collection

In [ ]:
! az cosmosdb mongodb collection list \
    -a $cosmosdbAccountName \
    -g $resourceGroup \
	-d $cosmosdbDatabaseName

#### 2.5.4. Add data to collection

1. Add data to Advertisements Collection - Connection String from 2.5.2

In [ ]:
# Rerun to get new connectionString after inserting
%run variables.sh

In [ ]:
! mongoimport \
    --uri=$connectionString \
    --db=$cosmosdbDatabaseName \
    --collection=advertisements \
    --file='./sample_data/sampleAds.json' \
    --jsonArray

2. Add data to Posts Collection - Connection String from 2.5.2

In [ ]:
! mongoimport \
    --uri=$connectionString \
    --db=$cosmosdbDatabaseName \
    --collection=posts \
    --file='./sample_data/samplePosts.json' \
    --jsonArray

In [ ]:
from pymongo import MongoClient

In [ ]:
# Create a MongoDB client
client = MongoClient($connectionString)

# Access the database and collection
db = client[f"neighborlydb"]

In [ ]:
advertisements = db["advertisements"]

result = advertisements.find({})

# Print out the query result
for doc in result:
    print(doc)


In [ ]:
posts = db["posts"]

result = posts.find({})

# Print out the query result
for doc in result:
    print(doc)


### 2.6. Create App Service for Front-End

1. List Runtime Environment

In [ ]:
! az webapp list-runtimes

2. Create App Service Plan

In [ ]:
! az appservice plan create \
    -n $appServicePlanName \
    -g $resourceGroup \
    -l $location \
    --sku $appSku \
    --is-linux

3. Create App Service

In [ ]:
! az webapp create \
    -n $appName \
    -g $resourceGroup \
    --plan $appServicePlanName \
    --runtime $appRuntime

## 3. Connect Function with CosmosDB

1. Verify the resources

In [ ]:
! az functionapp list  -g $resourceGroup

2. Connect

In [ ]:
! az functionapp config appsettings set \
  -n $functionName \
  -g $resourceGroup \
  --setting MyDbConnection="mongodb://neighborly-cosmosdb:7ZHd0B7QDvWXKK6WyjsSRh8n7a3A90og3WqdrklKOjb2XPTH1bEQcRKKCgzNLnf3IZDfUi0PHJXjACDbFmj7tA==@neighborly-cosmosdb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@neighborly-cosmosdb@"

## 4. Deploy Functions

1. Test Functions at Local

In [ ]:
! func start

2. Publish functions

In [ ]:
! func azure functionapp publish $functionName

## 5 Dockerize Functions and Deploy to AKS

### 5.1. Create a Dockerfile

In [ ]:
! func init --docker-only --python

### 5.2. Build the image using the Dockerfile

In [ ]:
! docker build -t $imageName:$imageTag .

In [ ]:
! docker run -p 7071:7071 -it $imageName:$imageTag

### 5.3. Create a repository in ACR service

1. Create Azure container registry

In [ ]:
! az acr create --resource-group $resourceGroup --name $containerRegistry --sku Basic

2. Login Azure container registry

In [ ]:
! az acr login --name $containerRegistry

3. Show container registry

In [ ]:
# Both these commands will give a same result
! az acr show --name $containerRegistry --query loginServer --output table
#! az acr list --resource-group $resourceGroup --query "[].{acrLoginServer:loginServer}" --output table

4. List docker images

In [ ]:
# List the local images
! docker images

5. Push image to Azure container registry

In [ ]:
# Tag the image with the same name as the ACR respository, else, the push will fail.
# SYNTAX 
# docker tag <name:tag> <ACR-respository>.azurecr.io/<name:tag>
! docker tag $imageName:$imageTag $containerRegistry.azurecr.io/$imageName:$imageTag

- Enable Admin User in Access Keys

In [ ]:
! az acr update --name $containerRegistry --admin-enabled true

- Verify Admin User Status

In [ ]:
! az acr show --name $containerRegistry --query "adminUserEnabled"

In [ ]:
# Go to Container Registry >> Settings >> Access Keys and enable the Admin user. 
# Use those credentials to login from your terminal. 
! docker login $containerRegistry.azurecr.io

In [ ]:
! docker push $containerRegistry.azurecr.io/$imageName:$imageTag

6. View Azure container registry image

In [ ]:
# View the newly pushed image in the ACR respository
! az acr repository list --name $containerRegistry --output table

### 5.3. Create an AKS cluster

1. Create an AKS cluster

In [ ]:
! az aks create \
    --name $aksCluster \
    --resource-group $resourceGroup \
    --node-count 2 \
    --generate-ssh-keys \
    --attach-acr $containerRegistry \
    --location $location

2. Get credentials for your container service

In [ ]:
# Run on terminal
! az aks get-credentials \
    -n $aksCluster \
    -g $resourceGroup

3. Verify the connection to your cluster and view the cluster nodes using

In [ ]:
! kubectl get nodes
#Example output:
#NAME                                STATUS   ROLES   AGE     VERSION
#aks-nodepool1-38114521-vmss000000   Ready    agent   3m47s   v1.21.9
#aks-nodepool1-38114521-vmss000001   Ready    agent   3m45s   v1.21.9

4. Deploy the App to Kubernetes

In [ ]:
! func kubernetes install --namespace keda

In [ ]:
# Run on terminal
! func kubernetes deploy \
    --name $functionName \
    --registry $containerRegistry.azurecr.io

In [ ]:
# Run on terminal
! kubectl apply -f ./deploy.yml

In [ ]:
! kubectl config get-contexts

## 6. Delete Resources

In [ ]:
! az group delete -n $resourceGroup --verbose